<a href="https://colab.research.google.com/github/saptab/inverse-scaling-2022-psych/blob/main/opt-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -r inverse-scaling-eval-pipeline/

# Set up
Run the cell below to install all of the necessary packages


In [ ]:
#@title Set up script { display-mode: "form" }
#@markdown Run this cell to install the necessary packages (may take a few minutes)
%%shell
cd /content
pip uninstall -y eval-pipeline
rm -rf public-colab
git clone -b public-colab --single-branch https://github.com/ed1d1a8d/inverse-scaling-eval-pipeline.git &> /dev/null
pip install git+https://github.com/ed1d1a8d/inverse-scaling-eval-pipeline.git@public-colab

# somehow these commands fix matplotlib to draw in the notebook
# https://stackoverflow.com/questions/64862818/cannot-import-name-png-from-matplotlib
%matplotlib inline
python -m pip uninstall matplotlib -y
pip install matplotlib==3.1.3 &> /dev/null


# Running

# An OPT Run
Choose from the evaluation metrics `classification`, `sequence_prob`, `logodds`, and `absolute_logodds`.

Descriptions of the evaluation metrics can be found in [this section of the README](https://github.com/inverse-scaling/prize).

To upload a file to the colab, click `Files` (the folder icon) on the sidebar on the left.
Then click `Upload to session storage` (the file with an arrow icon) and choose your `.csv` from your computer.

If you have Colab Pro or Colab Pro+, you can try running some larger OPT models in addition (select a High RAM runtime if possible).
Add `"opt-6.7b"` and `"opt-13b"` to the `model_names` list.
Unfortunately, the 30B and larger models are infeasible to run in a Colab environment.

(For those familiar with large language models: if you would like to run EleutherAI models, you can change the model_names to use models from `gpt-neo-125M`, `gpt-neo-1.3B`, `gpt-neo-2.7B`, and `gpt-j-6B`.)

In [ ]:
#@title Running OPT models and plotting the results { display-mode: "form" }
evaluation_metric = "classification" #@param ["classification", "sequence_prob", "logodds", "absolute_logodds", "classification_acc"]
file_name = "beck_hopelessness.csv" #@param {"type": "string"}
model_names = ["opt-125m", "opt-350m", "opt-1.3b", "opt-2.7b"] #@param {"type": "raw"}
model_names_string = ' '.join(model_names)

#@markdown Once you've specified an evaluation metric and a file name, run this cell
%run /content/inverse-scaling-eval-pipeline/eval_pipeline/main.py \
--dataset-path "/content/$file_name" \
--exp-dir /content/results \
--models $model_names_string \
--task-type $evaluation_metric \
--use-gpu

# we have to use %run because executing with !python does not load the python code in the colab shell
%run /content/inverse-scaling-eval-pipeline/eval_pipeline/plot_loss.py \
  --task-type $evaluation_metric \
  /content/results